This source is obtained from the web site of Jason Brownlee PhD,
Simple Genetic Algorithm From Scratch in Python,
https://machinelearningmastery.com/simple-genetic-algorithm-from-scratch-in-python/

Demonstration of binary-to-integer genetic algorithm.
For simplicity, we assume each integer parameter is encoded as 8-bit string ranging from 0~255


In [ ]:
# Min-Cost Flow Problem
from numpy.random import randint
from numpy.random import rand
import numpy as np

def mincostflow(x):
  unitcost = [700,300,200,400,500,1000]
  big_penalty = 100000

  # decode the binary string to interger values
  nintegers = len(x) // 8 # calculate the number of integer parameters
  intvalue = [0] * nintegers
  for i in range(nintegers):
    intvalue[i] = 0
    for j in range(8):
      intvalue[i] += x[i*8+j] * (2**(7-j))

  TotalCost = 0

  # compute the flow cost
  TotalCost = TotalCost + np.dot(intvalue,unitcost) # using inner product between two vectors

  # check if all capacity constraints are satisfied
  for i in range(4):
    if intvalue[i+1] > 50:
      TotalCost = TotalCost + (intvalue[i+1]-50)*big_penalty/10 # penalty if flow exceeds capacity

  # check if all net-flow constraints are satisfied
  if not (intvalue[0]+intvalue[1]==80):
    TotalCost = TotalCost + big_penalty # penalty if net-flow constraint is not respected
  if not (intvalue[4]+intvalue[5]==70):
    TotalCost = TotalCost + big_penalty # penalty if net-flow constraint is not respected
  if not (intvalue[0]+intvalue[2]==60):
    TotalCost = TotalCost + big_penalty # penalty if net-flow constraint is not respected
  if not (intvalue[3]+intvalue[5]==90):
    TotalCost = TotalCost + big_penalty # penalty if net-flow constraint is not respected

  return TotalCost


In [ ]:
# tournament selection
def selection(pop, scores, k=3):
	# first random selection
	selection_ix = randint(len(pop))
	for ix in randint(0, len(pop), k-1):
		# check if better (e.g. perform a tournament)
		if scores[ix] < scores[selection_ix]:
			selection_ix = ix
	return pop[selection_ix]

# crossover two parents to create two children
def crossover(p1, p2, r_cross):
	# children are copies of parents by default
	c1, c2 = p1.copy(), p2.copy()
	# check for recombination
	if rand() < r_cross:
		# select crossover point that is not on the end of the string
		pt = randint(1, len(p1)-2)
		# perform crossover
		c1 = p1[:pt] + p2[pt:]
		c2 = p2[:pt] + p1[pt:]
	return [c1, c2]

# mutation operator
def mutation(bitstring, r_mut):
	for i in range(len(bitstring)):
		# check for a mutation
		if rand() < r_mut:
			# flip the bit
			bitstring[i] = 1 - bitstring[i]

# binary-to-integer genetic algorithm
# for simplicity, we assume each integer parameter is encoded as 8-bit string ranging from 0~255

def genetic_algorithm(objective, n_bits, n_iter, n_pop, r_cross, r_mut):
	# initial population of random bitstring
	pop = [randint(0, 2, n_bits).tolist() for _ in range(n_pop)]
	# keep track of best solution
	best, best_eval = 0, objective(pop[0])
	# enumerate generations
	for gen in range(n_iter):
		# evaluate all candidates in the population
		scores = [objective(c) for c in pop]
		# check for new best solution
		for i in range(n_pop):
			if scores[i] < best_eval: # assume smaller the score the better
				best, best_eval = pop[i], scores[i]
				print(">%d, new best f(%s) = %.3f" % (gen,  pop[i], scores[i]))
		# select parents
		selected = [selection(pop, scores) for _ in range(n_pop)]
		# create the next generation
		children = list()
		for i in range(0, n_pop, 2):
			# get selected parents in pairs
			p1, p2 = selected[i], selected[i+1]
			# crossover and mutation
			for c in crossover(p1, p2, r_cross):
				# mutation
				mutation(c, r_mut)
				# store for next generation
				children.append(c)
		# replace population
		pop = children
	return [best, best_eval]

GA parameterization
[IMPORTANT] if your problem has k integer parameters, then n_bits = 8*k // 8 bits per integer ***

In [ ]:

# *** if your problem has k integer parameters, then n_bits = 8*k // 8 bits per integer ***
n_bits = 48  # IMPORTANT: we have SIX integer parameters in this example, customize it as you need

# define the total iterations
n_iter = 100
# define the population size
n_pop = 100
# crossover rate
r_cross = 0.9
# mutation rate
r_mut = 1.0 / float(n_bits)

Perform the GA to solve the problem

In [ ]:
# perform the genetic algorithm search
best, score = genetic_algorithm(mincostflow, n_bits, n_iter, n_pop, r_cross, r_mut)
print('Done!')
nintegers = len(best) // 8 # calculate the number of integer parameters
intvalue = [0]*nintegers
for i in range(nintegers):
  for j in range(8):
    intvalue[i] += best[i*8+j] * (2**(7-j))
  print('%d, ' % intvalue[i])
print('Min. cost = %f' % score)

>0, new best f([0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1]) = 2153100.000
>0, new best f([1, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 1, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0]) = 2120200.000
>0, new best f([1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1]) = 1778700.000
>0, new best f([1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 1]) = 593700.000
>1, new best f([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0]) = 546300.000
>2, new best f([0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 0, 